### Imports

In [7]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier

from imblearn.over_sampling import SMOTE
import pickle

import warnings 
warnings.filterwarnings('ignore')

# For reproducibility
np.random.seed(42)

In [2]:
!unzip datasets.zip

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [8]:
df = pd.read_csv('model_dataset.csv')

# We move the target variable to the front, for simplicity
df.insert(0, "accident_severity", df.pop("accident_severity"))

df.head()

,accident_severity,hour,lighting,intersection,atmosphere,collision,localisation,user_category,user_sex,pedestrian_action,road_category,traff_regime,longitud_profile,drawing_plan,surface_cond,acc_situation
0,Hospitalized wounded,14,Full day,Out of intersection,Cloudy weather,By the side,In built-up areas,Driver,Male,not specified,Departmental Road,Bidirectional,Dish,Curved right,normal,On the road
1,Hospitalized wounded,18,Full day,In intersection,Normal,Other,In built-up areas,Passenger,Female,not specified,Departmental Road,One way,Dish,Curved left,normal,On the road
2,Hospitalized wounded,19,Full day,Out of intersection,Normal,Other,Out of agglomeration,Pedestrian,Male,Opposite direction of the vehicle,Departmental Road,Bidirectional,Dish,Curved right,not normal,Off the road
3,Hospitalized wounded,19,Twilight or dawn,Out of intersection,Dazzling weather,By the side,In built-up areas,Driver,Male,not specified,Communal Way,Bidirectional,Dish,Straight part,normal,On the road
4,Hospitalized wounded,11,Full day,In intersection,Normal,By the side,In built-up areas,Passenger,Female,not specified,Communal Way,Bidirectional,Dish,Straight part,normal,On the road


### Encoding the features
Ideally we would use an nomical encoding technique like one-hot encoding, to avoid misleading our model. But given the amount of features and unique values, one-hot encoding might be more detrimental than beneficial, in terms of memory and computional power consumption 

In [9]:
# A dictionary containing the various encoders is stored, to be reused to deployment
encoders = {}

for column in df.columns:
    le = LabelEncoder()
    
    encoder = le.fit(df[column])
    df[column] = encoder.transform(df[column])
    
    encoders[column] = encoder
    
df.head()

,accident_severity,hour,lighting,intersection,atmosphere,collision,localisation,user_category,user_sex,pedestrian_action,road_category,traff_regime,longitud_profile,drawing_plan,surface_cond,acc_situation
0,0,14,0,1,0,0,0,0,1,7,1,0,0,1,0,1
1,0,18,0,0,5,4,0,1,0,7,1,1,0,0,0,1
2,0,19,0,1,5,4,1,2,1,3,1,0,0,1,1,0
3,0,19,4,1,1,0,0,0,1,7,0,0,0,3,0,1
4,0,11,0,0,5,0,0,1,0,7,0,0,0,3,0,1


In [12]:
with open('encoders.pkl', 'wb') as file:
  pickle.dump(encoders, file)

### Scaling and Train Test split

In [ ]:
X = df.drop(['accident_severity'], axis=1) 
Y = df['accident_severity']

scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

X.head()

,hour,lighting,intersection,atmosphere,collision,localisation,user_category,user_sex,pedestrian_action,road_category,traff_regime,longitud_profile,drawing_plan,surface_cond,acc_situation
0,0.081432,-0.631485,0.632882,-4.346320,-1.338037,-0.448257,-0.603092,0.721062,0.425910,0.261653,-0.615365,-0.435524,-1.702742,-0.484417,0.309949
1,0.820654,-0.631485,-1.580073,0.316759,0.541847,-0.448257,0.680063,-1.386843,0.425910,0.261653,0.749132,-0.435524,-2.770194,-0.484417,0.309949
2,1.005460,-0.631485,0.632882,0.316759,0.541847,2.230864,1.963217,0.721062,-1.251721,0.261653,-0.615365,-0.435524,-1.702742,2.064336,-3.226341
3,1.005460,2.473010,0.632882,-3.413704,-1.338037,-0.448257,-0.603092,0.721062,0.425910,-0.750008,-0.615365,-0.435524,0.432162,-0.484417,0.309949
4,-0.472985,-0.631485,-1.580073,0.316759,-1.338037,-0.448257,0.680063,-1.386843,0.425910,-0.750008,-0.615365,-0.435524,0.432162,-0.484417,0.309949


In [ ]:
Y.value_counts()

2    471695
0    321623
1     46667
Name: accident_severity, dtype: int64

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, stratify=Y)

### Modelling (initial)
We try out several well known models with their default hyperparameters<br>
Our key metrics are the
-  **macro** average F1-score 
-  recall for class 1(severity=killed)

In [ ]:
# !! IMPORTANT !!
# Double check the train and test dataset that's in memory before using the functions below 

def run_model_reports(model):
  """Fits model, makes prediction, and evaluates the result"""

  name = type(model).__name__
    
  # Fit the model
  print(f"Fitting {name} model...")
  model.fit(x_train, y_train)
        
  # Make predictions
  print("Making predictions...")
  y_pred = model.predict(x_test)
  
  # Evaluate metrics
  report = classification_report(y_test, y_pred)
  # The dictionary format is neccesary for extracting our key metrics
  report_dict = classification_report(y_test, y_pred, output_dict=True)

  return report, report_dict



def get_key_metrics(report_dict):
  """Extracts key metrics from the report"""

  report_df =  pd.DataFrame(report_dict)

  class_1_recall = report_df['1'].loc['recall']
  macro_f1 = report_df['macro avg'].loc['f1-score']

  return class_1_recall, macro_f1

In [ ]:
models = [LogisticRegression(n_jobs=-1),
          DecisionTreeClassifier(),
          RandomForestClassifier(n_jobs=-1),
          GradientBoostingClassifier(),
          AdaBoostClassifier(),
          XGBClassifier(n_jobs=-1),
          LGBMClassifier()]

In [ ]:
for m in models:
    report, report_dict = run_model_reports(m)
    recall, f1 = get_key_metrics(report_dict)
    
    print(report)
    print(f'Class 1 Recall: {round(recall, 4)}')
    print(f'Macro F1-Score: {round(f1, 4)}')
    print('-----------------------------------------------------')

Fitting LogisticRegression model...
Making predictions...
              precision    recall  f1-score   support

           0       0.63      0.37      0.46     96487
           1       0.60      0.00      0.00     14000
           2       0.66      0.91      0.77    141509

    accuracy                           0.65    251996
   macro avg       0.63      0.43      0.41    251996
weighted avg       0.65      0.65      0.61    251996

Class 1 Recall: 0.0002
Macro F1-Score: 0.4109
-----------------------------------------------------
Fitting DecisionTreeClassifier model...
Making predictions...
              precision    recall  f1-score   support

           0       0.54      0.47      0.50     96487
           1       0.17      0.09      0.12     14000
           2       0.68      0.77      0.73    141509

    accuracy                           0.62    251996
   macro avg       0.46      0.44      0.45    251996
weighted avg       0.60      0.62      0.61    251996

Class 1 Recall: 0.

### Resampling
From the evaluation above, the decision tree model had the best recall for class 1 (score of 0.09), and most of the models achieved a macro average F1-score of around 0.44.<br>
These are poor metrics, and are due to the imbalanced dataset, thus we proceeded to upsample our minority classes: 1(killed) and class 0(hospitalized)

In [ ]:
sm = SMOTE(sampling_strategy='all', n_jobs=-1)
resampled_X, resampled_Y = sm.fit_resample(X, Y)

# Check the new class distribution
resampled_Y.value_counts()

0    471695
2    471695
1    471695
Name: accident_severity, dtype: int64

In [ ]:
resampled_df = pd.concat([resampled_Y, resampled_X], axis=1)
# Shuffle the dataset
resampled_df = resampled_df.sample(frac=1).reset_index(drop=True)

To avoid re-running the SMOTE resample function, the dataset was saved to a csv file

In [ ]:
resampled_df.to_csv('resampled_dataset.csv', index=False)

In [ ]:
resampled_df = pd.read_csv('resampled_dataset.csv')
resampled_df.head()

,accident_severity,hour,lighting,intersection,atmosphere,collision,localisation,user_category,user_sex,pedestrian_action,road_category,traff_regime,longitud_profile,drawing_plan,surface_cond,acc_situation
0,2,0.635849,-0.631485,-1.580073,0.316759,-1.338037,-0.448257,-0.603092,0.721062,0.425910,-0.750008,-0.615365,2.426502,0.432162,-0.484417,0.309949
1,1,-2.264236,1.696886,0.632882,0.316759,0.071876,2.230864,-0.603092,0.721062,0.425910,0.261653,-0.615365,-0.435524,0.432162,-0.484417,0.309949
2,0,-1.235226,-0.631485,0.632882,0.316759,0.541847,2.230864,1.963217,0.721062,-2.509945,2.284975,-0.615365,-0.435524,0.432162,-0.484417,0.309949
3,0,0.635849,-0.631485,0.632882,0.316759,-1.338037,-0.448257,-0.603092,0.721062,0.425910,-0.750008,-0.615365,-0.435524,0.432162,-0.484417,0.309949
4,0,-0.103374,-0.631485,-1.580073,-0.615857,-1.338037,-0.448257,0.680063,-1.386843,0.425910,-0.750008,-0.615365,-0.435524,0.432162,2.064336,0.309949


### Train test split on the new resampled dataset

In [ ]:
resampled_X = resampled_df.drop(['accident_severity'], axis=1) 
resampled_Y = resampled_df['accident_severity']

x_train, x_test, y_train, y_test = train_test_split(resampled_X, resampled_Y, test_size=0.3)

### Modelling (second)

In [ ]:
for m in models:
    report, report_dict = run_model_reports(m)
    recall, f1 = get_key_metrics(report_dict)
    
    print(report)
    print(f'Class 1 Recall: {round(recall, 4)}')
    print(f'Macro F1-Score: {round(f1, 4)}')
    print('-----------------------------------------------------')

Fitting LogisticRegression model...
Making predictions...
              precision    recall  f1-score   support

           0       0.39      0.17      0.24    141522
           1       0.58      0.62      0.60    141596
           2       0.52      0.78      0.63    141408

    accuracy                           0.52    424526
   macro avg       0.50      0.52      0.49    424526
weighted avg       0.50      0.52      0.49    424526

Class 1 Recall: 0.6153
Macro F1-Score: 0.4894
-----------------------------------------------------
Fitting DecisionTreeClassifier model...
Making predictions...
              precision    recall  f1-score   support

           0       0.53      0.46      0.50    141522
           1       0.79      0.79      0.79    141596
           2       0.62      0.71      0.66    141408

    accuracy                           0.65    424526
   macro avg       0.65      0.65      0.65    424526
weighted avg       0.65      0.65      0.65    424526

Class 1 Recall: 0.

We can see a great inprovement in our metrics after resampling<br>
Our best model so far is the Random Forest model (Class 1 Recall: 0.822
Macro F1-Score: 0.6715)<br>
However we were still not satisfied with it's performance, our next approach was to select our top 3 models and tune thier hyperparamters to obtain better results.<br>
This time our key metrics is to improve **accuracy** and the **recall for class 0**

### Hyperparameter tunning
The best performing models were trained using a random search CV

In [ ]:
def randomsearch_cv(model, grid, cv, n_iter):
  """Performs the random search and returns the best model"""

  rs_cv = RandomizedSearchCV(estimator=model, param_distributions=grid, cv=cv,
                             n_iter=n_iter, scoring="accuracy", n_jobs=-1, verbose=1)
  
  rs_cv.fit(x_train, y_train)
  
  print(f'Best hyperparameters: {rs_cv.best_params_}')

  return rs_cv.best_estimator_

- Random Forest

In [ ]:
# Define model and hyperparameters
rfc = RandomForestClassifier(n_jobs=-1)
rfc_grid = {'n_estimators': [50, 100, 200, 500, 1000],
            'max_features': ['auto', 'sqrt', 'log2', 'none']}

In [ ]:
# Fit the model with the best hyperparameters
rfc_best = randomsearch_cv(rfc, rfc_grid, 2, 2)

rfc_best.fit(x_train, y_train)
y_pred_rfc = rfc_best.predict(x_test)

Fitting 2 folds for each of 2 candidates, totalling 4 fits
Best hyperparameters: {'n_estimators': 100, 'max_features': 'log2'}


In [ ]:
# Evaluate the model
report = classification_report(y_test, y_pred_rfc)
print(report)

              precision    recall  f1-score   support

           0       0.58      0.47      0.52    140988
           1       0.81      0.82      0.82    141628
           2       0.63      0.74      0.68    141910

    accuracy                           0.68    424526
   macro avg       0.67      0.68      0.67    424526
weighted avg       0.67      0.68      0.67    424526



- Decision Tree

In [ ]:
models[1].tree_.max_depth

In [ ]:
# Define model and hyperparameters
dtc = DecisionTreeClassifier()
dtc_grid = {'criterion': ['gini', 'entropy'],
            'splitter': ['best', 'random'],
            'max_features': ['auto', 'sqrt', 'log2', 'none'], 
            'max_depth': [20, 40, None]}

In [ ]:
# Fit the model with the best hyperparameters
dtc_best = randomsearch_cv(dtc, dtc_grid, 5, 10)

dtc_best.fit(x_train, y_train)
y_pred_dtc = dtc_best.predict(x_test)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best hyperparameters: {'splitter': 'best', 'max_features': 'auto', 'max_depth': None, 'criterion': 'entropy'}


In [ ]:
# Evaluate the model
report = classification_report(y_test, y_pred_dtc)
print(report)

              precision    recall  f1-score   support

           0       0.54      0.47      0.50    140988
           1       0.79      0.79      0.79    141628
           2       0.62      0.70      0.66    141910

    accuracy                           0.65    424526
   macro avg       0.65      0.65      0.65    424526
weighted avg       0.65      0.65      0.65    424526



- LightGBM

In [ ]:
# Define model and hyperparameters
lgbmc = LGBMClassifier()
lgbmc_grid = {'num_leaves': [6, 10, 20, 30, 50],
              'min_child_samples': [100, 200, 300, 500],
              'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4]}

In [ ]:
# Fit the model with the best hyperparameters
lgbmc_best = randomsearch_cv(lgbmc, lgbmc_grid, 3, 3)

lgbmc_best.fit(x_train, y_train)
y_pred_lgbmc = lgbmc_best.predict(x_test)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
Best hyperparameters: {'num_leaves': 50, 'min_child_weight': 1000.0, 'min_child_samples': 500}


In [ ]:
# Evaluate the model
report = classification_report(y_test, y_pred_lgbmc)
print(report)

              precision    recall  f1-score   support

           0       0.53      0.37      0.44    140988
           1       0.76      0.78      0.77    141628
           2       0.63      0.79      0.70    141910

    accuracy                           0.65    424526
   macro avg       0.64      0.65      0.64    424526
weighted avg       0.64      0.65      0.64    424526



### Voting Classifier
Finally, to properly utilize each model's strenght, and decrease the overall error, we used a voting classifier to combine our three models

In [ ]:
voting_clf = VotingClassifier(estimators=[('rfc', rfc_best), ('dtc', dtc_best), ('lgbmc', lgbmc_best)],
                              voting='hard')

voting_clf.fit(x_train, y_train)

VotingClassifier(estimators=[('rfc',
                              RandomForestClassifier(max_features='log2',
                                                     n_jobs=-1)),
                             ('dtc',
                              DecisionTreeClassifier(criterion='entropy',
                                                     max_features='auto')),
                             ('lgbmc',
                              LGBMClassifier(min_child_samples=500,
                                             min_child_weight=1000.0,
                                             num_leaves=50))])

In [ ]:
y_pred = voting_clf.predict(x_test)
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.57      0.47      0.51    140988
           1       0.81      0.82      0.81    141628
           2       0.64      0.74      0.68    141910

    accuracy                           0.68    424526
   macro avg       0.67      0.68      0.67    424526
weighted avg       0.67      0.68      0.67    424526



Although the voting classier model had some metrics that were lower than those of some individual model, it was selected as our final model because it had a more evenly distributed scores. 

### Save model to pickel file

In [ ]:
with open('model.pkl', 'wb') as file:
  pickle.dump(voting_clf, file)

In [ ]:
!du -sh 'model.pkl'

4.3G	model.pkl


Our model was quite large, which would be difficult to hanlde in deployment.<br>
We also tried several approaches to reduced the file size; using joblib, removing the random forest model, zipping the file<br> Regardless, the file was still too large, thus we resulted in using only the decision tree for deployment

In [ ]:
with open('dtc_model.pkl', 'wb') as file:
  pickle.dump(dtc_best, file)

In [ ]:
!du -sh 'dtc_model.pkl'

30M	dtc_model.pkl
